In [20]:
#https://medium.com/swlh/using-q-learning-for-openais-cartpole-v1-4a216ef237df
#source from 
#A extension on the project
import numpy as np # used for arrays

import gym # pull the environment

import time # to get the time

import math # needed for calculations

import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

In [21]:
import pyvirtualdisplay


_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

In [22]:
env = gym.make("CartPole-v1")
print(env.action_space.n)

2


In [23]:
LEARNING_RATE = 0.1

DISCOUNT = 0.95
EPISODES = 60000
total = 0
total_reward = 0
prior_reward = 0

Observation = [30, 30, 50, 50]
np_array_win_size = np.array([0.25, 0.25, 0.01, 0.1])

epsilon = 1

epsilon_decay_value = 0.99995

In [24]:
q_table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))
q_table.shape

(30, 30, 50, 50, 2)

In [25]:
def get_discrete_state(state):
    discrete_state = state/np_array_win_size+ np.array([15,10,1,10])
    return tuple(discrete_state.astype(np.int))

In [26]:
def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s | Step: %d %s" % (env._spec.id,step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

In [33]:
for episode in range(EPISODES + 1): #go through the episodes
    t0 = time.time() #set the initial time
    discrete_state = get_discrete_state(env.reset()) #get the discrete start for the restarted environment 
    done = False
    episode_reward = 0 #reward starts as 0 for each episode

    if episode % 2000 == 0: 
        print("Episode: " + str(episode))

    while not done: 

        if np.random.random() > epsilon:

            action = np.argmax(q_table[discrete_state]) #take cordinated action
        else:

            action = np.random.randint(0, env.action_space.n) #do a random ation

        new_state, reward, done, _ = env.step(action) #step action to get new states, reward, and the "done" status.

        episode_reward += reward #add the reward

        new_discrete_state = get_discrete_state(new_state)

        #if episode % 2000 == 0: #render
            #env.render()

        if not done: #update q-table
            max_future_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action,)]

            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state

    if epsilon > 0.05: #epsilon modification
        if episode_reward > prior_reward and episode > 10000:
            epsilon = math.pow(epsilon_decay_value, episode - 10000)

            if episode % 500 == 0:
                print("Epsilon: " + str(epsilon))

    t1 = time.time() #episode has finished
    episode_total = t1 - t0 #episode total time
    total = total + episode_total

    total_reward += episode_reward #episode total reward
    prior_reward = episode_reward

    if episode % 1000 == 0: #every 1000 episodes print the average time and the average reward
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = total_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        total_reward = 0

env.close()

Episode: 0
Time Average: 2.5198459625244143e-06
Mean Reward: 0.016
Time Average: 0.0013300015926361083
Mean Reward: 22.272
Episode: 2000
Time Average: 0.0013951184749603272
Mean Reward: 22.635
Time Average: 0.0013160240650177003
Mean Reward: 22.296
Episode: 4000
Time Average: 0.0012665488719940186
Mean Reward: 22.118
Time Average: 0.0013047440052032472
Mean Reward: 21.842
Episode: 6000
Time Average: 0.0013705005645751954
Mean Reward: 22.792
Time Average: 0.0013414056301116943
Mean Reward: 22.999
Episode: 8000
Time Average: 0.0013695015907287597
Mean Reward: 21.805
Time Average: 0.001259756088256836
Mean Reward: 22.175
Episode: 10000
Time Average: 0.0013080518245697022
Mean Reward: 22.718
Time Average: 0.0013596465587615966
Mean Reward: 22.715
Episode: 12000
Time Average: 0.0013784351348876953
Mean Reward: 23.797
Epsilon: 0.8824941446941661
Epsilon: 0.8607047486686201
Time Average: 0.0015139980316162109
Mean Reward: 25.225
Episode: 14000
Time Average: 0.0015770161151885986
Mean Reward: 

KeyboardInterrupt: ignored